# 04. Un scraper complet

Dans ce cours:
- Créer des fonctions
- Gérer le tempo de ses requêtes
- Utiliser des boucles pour parser plusieurs pages
- Extraires des données depuis une API
- Déguiser son scraper

In [ ]:
# Résumé du cours précédent

# Bonne pratique: Mettre tous ses imports au début du code
import requests, csv
from bs4 import BeautifulSoup

url = "https://www.leboncoin.fr/materiel_agricole/"
r = requests.get(url)

# Bonne pratique: Ajouter des conditions pour prendre en compte les erreurs possibles
if r.status_code == 200:
    soup = BeautifulSoup(r.text, 'html.parser')
else:
    raise Exception("Une erreur est survenue lors de la requête de la page %s" % url)

# Extraction des annonces de la page HTML
annonces = soup.find("section", {"class": "tabsContent"}).find_all("a", {"class": "list_item"})

data = []

# Parse les annonces
for annonce in annonces:
    titre =annonce.find("h2").text.strip()
    prix = annonce.find("h3")
    if (prix != None):
        prix = prix.text.strip()
    lieu = annonce.find("p", {"itemtype": "http://schema.org/Place"}).text.strip()
    commune = lieu.split("/")[0].strip()
    département = lieu.split("/")[1].strip()
    
    item = {"titre": titre, "commune": commune, "département": département, "prix": prix}
    
    data.append(item)

# Enregistrement des données au format CSV
keys = data[0].keys()
with open('data_agricole.csv', 'w') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(data)
    
# Bonne pratique: Ajouter une ligne résumant ce que le programme vient de faire
print("Nous venons de scraper %s annonces." % len(data))

## Les fonctions

Une fonction est un sous-programme utilisé dans un programme principal. Elle peut être appellée plusieurs fois dans le même programme. 

Elle permet de rendre le code plus lisible et plus court. Nous avons déjà utilisé plusieurs _méthodes_ (strip(), append(), find_all() etc.), qui sont presque comme des fonctions (ce sont les fonctions d'un _objet_). 

Une fonction commence toujours par:

`def nom_de_la_fonction():`

Elle peut accepter des arguments:

`def calculDuBMI(poids, taille):`

Elle peut retourner une valeur:

```def calculduBMI(poids, taille):
    bmi = poids / (taille**2)
    return bmi ```
    
On doit toujours écrire la fonction **avant** de l'utiliser dans son code.

In [ ]:
def calculduBMI(poids, taille):
    bmi = poids / (taille**2) # L'opération ** est un exposant. x**2 est la même chose que x². x**3 équivaut à x³
    return bmi

mon_bmi = calculduBMI(60, 1.70)

print (mon_bmi)

In [ ]:
# Exercice: Calculer le BMI de toutes les personnes de la liste

patients = [
    {"name": "Louise", "taille": 160, "poids": 45},
    {"name": "Emma", "taille": 164, "poids": 63},
    {"name": "Lola", "taille": 171, "poids": 62},
    {"name": "Aaron", "taille": 180, "poids": 90},
    {"name": "Adam", "taille": 168, "poids": 55},
    {"name": "Anaël", "taille": 162, "poids": 58}
]

"""
    Ecrivez un programme utilisant la fonction calculduBMI() qui écrive, pour chaque patient: 
        Louise mesure 1,60 mètre et pèse 45 kg. Son IMC est de 17.6.
    
    Pour formatter un nombre avec une décimale après la virgule, on utilise %.1f.
    Ex: print("Son IMC est de %.1f." % imc)
"""

In [ ]:
# Excercice: Créer une fonction qui calcule, 
# pour chaque couple possible dans la liste, 
# un score de compatibilité des prénoms
# Le score est comptabilisé de la manière suivante:
# Sur la base des deux prénoms mis bout à bout:
# Un point pour chaque voyelle
# 5 points si il y a un M
# -5 points si il y a un S

# Pour vérifier si une chaîne de caractère est dans une autre chaîne de caractères,
# on utilise "in":
# Ex: if "a" in "Emma"

prénoms = ["Louise", "Emma", "Lola", "Aaron", "Adam", "Anaël"]

# Ecrire la fonction score() ici!

for nom1 in prénoms:
    for nom2 in prénoms:
        if (nom1 != nom2):
            print ("Score de compatibilité entre %s et %s: %s" % (nom1, nom2, score(nom1, nom2)))

In [ ]:
# Réécriture de notre code avec des fonctions

# Pour gérer le tempo des requêtes, on a besoin de deux nouveaux modules
import requests, csv, time, random
from bs4 import BeautifulSoup

def getPage(url):
    # On ajoute un intervalle aléatoire entre 1 et 2 secondes entre chaque requête
    # La fonction sleep() suspend le programme pour une durée en seconde. La fonction random()
    # retourne un nombre entre 0 et 1.
    time.sleep(1 + random.random())
    r = requests.get(url)
    if r.status_code == 200:
        soup = BeautifulSoup(r.text, 'html.parser')
    else:
        raise Exception("Une erreur est survenue lors de la requête de la page %s" % url)
    return soup

def parseAnnonces(annonce):
    titre =annonce.find("h2").text.strip()
    prix = annonce.find("h3")
    if (prix != None):
        prix = prix.text.strip()
    lieu = annonce.find("p", {"itemtype": "http://schema.org/Place"}).text.strip()
    commune = lieu.split("/")[0].strip()
    # Certaines annonces n'ont pas de département ou de commune
    # Il faut prendre en compte cette possibilité
    département = ""
    if (len(lieu.split("/"))>1):
        département = lieu.split("/")[1].strip()
    item = {"titre": titre, "commune": commune, "département": département, "prix": prix}
    return item

def saveCSV(data, filename):
    keys = data[0].keys()
    with open(filename, 'w') as output_file:
        dict_writer = csv.DictWriter(output_file, keys)
        dict_writer.writeheader()
        dict_writer.writerows(data)

url = "https://www.leboncoin.fr/materiel_agricole/"

soup = getPage(url)

# Extraction des annonces de la page HTML
annonces = soup.find("section", {"class": "tabsContent"}).find_all("a", {"class": "list_item"})

data = []

# Parse les annonces
for annonce in annonces:
    item = parseAnnonces(annonce)
    data.append(item)

# Enregistrement des données au format CSV
saveCSV(data, "materiel_agricole.csv")

# Bonne pratique: Ajouter une ligne résumant ce que le programme vient de faire
print("Nous venons de scraper %s annonces." % len(data))

In [ ]:
# Quand on travaille avec des boucles, il faut bien faire attention à l'endroit où on
# initialise ses variable. Si on initialisait data au sein de la boucle, la variable
# se remettrait à 0 à chaque nouvelle page. Or, on veut qu'elle contienne les données
# de toutes les pages.
data = []

# Python permet de créer des ranges, qui se comportent presque comme des listes
# Ici, c'est presque comme si on avait une liste [1,2,3,4,5]
# Je dis presque car toutes les méthodes des listes ne sont pas disponibles pour les ranges.
for page in range(1,5):
    # Le raccourci %d indique à Python que l'on va insérer un nombre entier dans cette chaîne de caractères
    url = "https://www.leboncoin.fr/materiel_agricole/offres/?o=%d" % page
    
    soup = getPage(url)
    
    # Bonne pratique: ajouter des textes qui indique ce que le programme est en train de faire.
    print("Le programme scrape la page numéro %d" % page)

    # Extraction des annonces de la page HTML
    annonces = soup.find("section", {"class": "tabsContent"}).find_all("a", {"class": "list_item"})

    # Parse les annonces
    for annonce in annonces:
        item = parseAnnonces(annonce)
        data.append(item)

# Enregistrement des données au format CSV
saveCSV(data, "materiel_agricole_5pages.csv")

# Bonne pratique: Ajouter une ligne résumant ce que le programme vient de faire
print("Nous venons de scraper %s annonces." % len(data))

## Les API

Une API ou interface de programmation permet de faire une requête à un serveur, qui nous renvoit des données structurées (et non pas une page HTML).

En général, les API sont documentées.

On utilise ici une API pour géolocaliser nos annonces: [Nominatim](https://wiki.openstreetmap.org/wiki/Nominatim).

Voilà la requête à effectuer pour obtenir la géolocalisation de Berlin, par exemple:

https://nominatim.openstreetmap.org/search/?format=json&q=Berlin&countrycodes=DE&limit=1

Ou Agen:

https://nominatim.openstreetmap.org/search/?format=json&q=Agen,Lot-et-Garonne&countrycodes=FR&limit=1

Les APIs utilisent la notation JSON, qui ressemble fort à des listes et des dictionnaires Python.

Pour visualiser le JSON dans le navigateur, utilisez un plugin comme [JSON View](https://addons.mozilla.org/en-US/firefox/addon/jsonview/?src=recommended) pour Firefox.

In [ ]:
# Un module pour convertir le format JSON en dicts et listes
import json

# Une fonction pour géolocaliser une adresse

def geoloc(adresse):
    url = "https://nominatim.openstreetmap.org/search/?format=json&q=%s&countrycodes=FR&limit=1" % adresse
    time.sleep(1 + random.random())
    r = requests.get(url)
    # On dit à Python de parser le JSON
    data = json.loads(r.text)
    # On regarde le premier élement de la liste
    result = data[0]
    # On regarde les valeurs de lat et lon dans le dict
    lat = result["lat"]
    lon = result["lon"]
    
    # Une fonction peut retourner plusieurs valeurs
    return lat, lon

lat, lon = geoloc("Chambéry,Savoie")

print ("Les coordonnées de Chambéry sont %s de latitude et %s de longitude" % (lat, lon))

In [ ]:
# Exercice: Utiliser la fonction geoloc pour scraper les 5 premières page de Le Bon Coin (catégorie au choix)
# et produire un fichier CSV contenant les coordonnées lat et lon de chaque annonce.